In [1]:
if 0 :
    %matplotlib inline
else :
    %matplotlib notebook

#  Import  libraries

In [2]:
import sys
import os
module_path = os.path.abspath('.') +"\\_scripts"
print(module_path)

if module_path not in sys.path:
    sys.path.append(module_path)
from _00_Import_packages_git import *

C:\Users\ng947ac\jupyter\SoSTrades\jupyter_doc\_scripts
C:\Users\ng947ac\jupyter\git\workspace
C:\Users\ng947ac\jupyter\git\workspace\gems
C:\Users\ng947ac\jupyter\SoSTrades\SoSTrade\pycryptodome-master
C:\Users\ng947ac\jupyter\git\workspace\sos_trades_core


In [3]:
from numpy import array
import pandas as pd
from sos_trades_core.execution_engine.execution_engine import ExecutionEngine
from numpy.testing import assert_array_equal, assert_array_almost_equal  # @UnresolvedImport
import os
from gemseo.core.mdo_scenario import MDOScenario

# SoSDoeScenario setUp

In [4]:
study_name = 'doe'
ns = f'{study_name}'
sc_name = "DiscAllTypesDoeScenario"
c_name = "DiscAllTypesCoupling"
print(sc_name)

DiscAllTypesDoeScenario


See §2 for Discipmline description

In [5]:
dspace_dict = {'variable': ['z', 'h'],
               'value': [1.,[5.,2.]],
               'lower_bnd': [0., [-10.,0.]],
               'upper_bnd': [10., [10.,10.]],
               'enable_variable': [True, True],
               'activated_elem': [[True], [True, True]]}
#                   'type' : ['float',['float','float'],'float','float']

In [6]:
dspace = pd.DataFrame(dspace_dict)
dspace

,variable,value,lower_bnd,upper_bnd,enable_variable,activated_elem
0,z,1.0,0.0,10.0,True,[True]
1,h,"[5.0, 2.0]","[-10.0, 0.0]","[10.0, 10.0]",True,"[True, True]"


In [7]:
from tempfile import gettempdir
root_dir = gettempdir()
print(root_dir)
dspace.to_csv(root_dir+'\\ds.csv')

C:\Temp


In [8]:
repo = 'sos_trades_core.sos_processes.test'
proc_name = 'test_DiscAllTypes_doe'

# Doe scenario execution

# 1 Set Doe optim scenario lhs or fullfact

In [9]:
#my_formulation = "DisciplinaryOpt"
my_formulation = "MDF"
#my_formulation = "IDF"
#my_formulation =  "BiLevel "

In [10]:
from gemseo.api import (
    configure_logger,
    get_algorithm_options_schema,
    get_available_doe_algorithms,
)
get_available_doe_algorithms()

['CustomDOE',
 'DiagonalDOE',
 'OT_SOBOL',
 'OT_RANDOM',
 'OT_HASELGROVE',
 'OT_REVERSE_HALTON',
 'OT_HALTON',
 'OT_FAURE',
 'OT_MONTE_CARLO',
 'OT_FACTORIAL',
 'OT_COMPOSITE',
 'OT_AXIAL',
 'OT_OPT_LHS',
 'OT_LHS',
 'OT_LHSC',
 'OT_FULLFACT',
 'OT_SOBOL_INDICES',
 'fullfact',
 'ff2n',
 'pbdesign',
 'bbdesign',
 'ccdesign',
 'lhs']

In [11]:
my_doe_algo = "lhs"
#my_doe_algo = "fullfact"

In [12]:
print("\n DiscAllTypes doe solution check with ",my_formulation ," formulation")


 DiscAllTypes doe solution check with  MDF  formulation


In [13]:
exec_eng = ExecutionEngine(study_name)
factory = exec_eng.factory
builder = factory.get_builder_from_process(repo=repo,mod_id=proc_name)
exec_eng.factory.set_builders_to_coupling_builder(builder)
exec_eng.configure()

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi


In [14]:
exec_eng.display_treeview_nodes()

INFO:SoS.EE:Nodes representation for Treeview doe
|_ doe
	|_ DiscAllTypesDoeScenario
		|_ DiscAllTypes


'Nodes representation for Treeview doe\n|_ doe\n\t|_ DiscAllTypesDoeScenario\n\t\t|_ DiscAllTypes'

In [15]:
exec_eng.dm.get_data('doe.sub_mda_class')['value']

'MDAJacobi'

Possible algo options:

In [16]:
from gemseo.algos.doe.doe_factory import DOEFactory
driver_lib = DOEFactory().create(my_doe_algo)
driver_lib.init_options_grammar(my_doe_algo)
algo_options_keys = list(driver_lib.opt_grammar.properties_dict.keys())
print(algo_options_keys)

['criterion', 'iterations', 'n_samples', 'eval_jac', 'n_processes', 'wait_time_between_samples', 'seed', 'max_time']


In [17]:
#-- set up disciplines in Scenario
disc_dict = {}
# Doe inputs
disc_dict[f'{ns}.DiscAllTypesDoeScenario.n_samples'] = 100
# 'lhs', 'CustomDOE', 'fullfact', ...
disc_dict[f'{ns}.DiscAllTypesDoeScenario.algo'] = 'lhs'
disc_dict[f'{ns}.DiscAllTypesDoeScenario.design_space'] = dspace

disc_dict[f'{ns}.DiscAllTypesDoeScenario.formulation'] = 'MDF'
disc_dict[f'{ns}.DiscAllTypesDoeScenario.objective_name'] = 'o'
#disc_dict[f'{ns}.DiscAllTypesDoeScenario.ineq_constraints'] = [f'c_1', f'c_2']
#disc_dict[f'{ns}.DiscAllTypesDoeScenario.algo_options'] = {'levels': 'None'
#                                                    }

disc_dict

{'doe.DiscAllTypesDoeScenario.n_samples': 100,
 'doe.DiscAllTypesDoeScenario.algo': 'lhs',
 'doe.DiscAllTypesDoeScenario.design_space':   variable       value     lower_bnd     upper_bnd  enable_variable  \
 0        z         1.0           0.0          10.0             True   
 1        h  [5.0, 2.0]  [-10.0, 0.0]  [10.0, 10.0]             True   
 
   activated_elem  
 0         [True]  
 1   [True, True]  ,
 'doe.DiscAllTypesDoeScenario.formulation': 'MDF',
 'doe.DiscAllTypesDoeScenario.objective_name': 'o'}

Remark: in fact it is a <b> DoE optim </b> (with 'obj' variable taken as obj function) ! Here dspace variable was used.

In [18]:
my_dict = exec_eng.get_anonimated_data_dict()
pd.DataFrame.from_dict(disc_dict,orient='index')

,0
doe.DiscAllTypesDoeScenario.n_samples,100
doe.DiscAllTypesDoeScenario.algo,lhs
doe.DiscAllTypesDoeScenario.design_space,variable value lower_bnd upper...
doe.DiscAllTypesDoeScenario.formulation,MDF
doe.DiscAllTypesDoeScenario.objective_name,o


In [19]:
exec_eng.dm.set_values_from_dict(disc_dict)

# 2 Display Disciplines I/O variables

In [20]:
disc = exec_eng.dm.get_disciplines_with_name("doe." + "DiscAllTypesDoeScenario.DiscAllTypes")[0]

In [21]:
type(disc)

sos_trades_core.sos_disciplines.test_discs.disc_all_types.DiscAllTypes

In [22]:
DESC_IN = disc.DESC_IN
DESC_OUT = disc.DESC_OUT
DESC_IN_df = pd.DataFrame.from_dict(DESC_IN,orient='index')
DESC_OUT_df = pd.DataFrame.from_dict(DESC_OUT,orient='index')

In [23]:
DESC_IN_df

,type,default,unit,user_level,visibility,namespace
z,float,90.0,kg,1,Shared,ns_test
h,array,NaN,kg,1,NaN,NaN
dict_in,dict,NaN,kg,1,NaN,NaN
df_in,dataframe,NaN,kg,1,NaN,NaN
weather,string,"cloudy, it is Toulouse ...",NaN,1,Shared,ns_test
dict_of_dict_in,dict,NaN,NaN,1,NaN,NaN
dict_of_df_in,dict,NaN,NaN,1,NaN,NaN


In [24]:
DESC_OUT_df

,type,unit,user_level,visibility,namespace
df_out,dataframe,kg,1,Shared,ns_test
o,array,kg,1,Shared,ns_test
dict_out,dict,kg,1,Shared,ns_test


# 3 Provide Discipline inputs

In [25]:
h_data = array([0., 0., 0., 0.])
h_data

array([0., 0., 0., 0.])

In [26]:
dict_in_data = {'key0': 0., 'key1': 0.}
dict_in_data

{'key0': 0.0, 'key1': 0.0}

In [27]:
df_in_data = pd.DataFrame(array([[0.0, 1.0, 2.0], [0.1, 1.1, 2.1],
                             [0.2, 1.2, 2.2], [-9., -8.7, 1e3]]),
                            columns=['variable', 'c2', 'c3'])
df_in_data

,variable,c2,c3
0,0.0,1.0,2.0
1,0.1,1.1,2.1
2,0.2,1.2,2.2
3,-9.0,-8.7,1000.0


In [28]:
weather_data = 'cloudy, it is Toulouse ...'

In [29]:
dict_of_dict_in_data = {'key_A': {'subKey1': 0.1234, 'subKey2': 111.111, 'subKey3': 2036},
                         'key_B': {'subKey1': 1.2345, 'subKey2': 222.222, 'subKey3': 2036}}
dict_of_dict_in_data

{'key_A': {'subKey1': 0.1234, 'subKey2': 111.111, 'subKey3': 2036},
 'key_B': {'subKey1': 1.2345, 'subKey2': 222.222, 'subKey3': 2036}}

In [30]:
a_df = pd.DataFrame(array([[5., -.05, 5.e5, 5.**5], [2.9, 1., 0., -209.1],
                    [0.7e-5, 2e3 / 3, 17., 3.1416], [-19., -2., -1e3, 6.6]]),
                     columns=['key1', 'key2', 'key3', 'key4'])
a_df

,key1,key2,key3,key4
0,5.000000,-0.050000,500000.0,3125.0000
1,2.900000,1.000000,0.0,-209.1000
2,0.000007,666.666667,17.0,3.1416
3,-19.000000,-2.000000,-1000.0,6.6000


In [31]:
dict_of_df_in_data = {'key_C': a_df,
                      'key_D': a_df * 3.1416}
dict_of_df_in_data 

{'key_C':         key1        key2      key3       key4
 0   5.000000   -0.050000  500000.0  3125.0000
 1   2.900000    1.000000       0.0  -209.1000
 2   0.000007  666.666667      17.0     3.1416
 3 -19.000000   -2.000000   -1000.0     6.6000,
 'key_D':         key1        key2          key3         key4
 0  15.708000    -0.15708  1.570800e+06  9817.500000
 1   9.110640     3.14160  0.000000e+00  -656.908560
 2   0.000022  2094.40000  5.340720e+01     9.869651
 3 -59.690400    -6.28320 -3.141600e+03    20.734560}

In [32]:
dict_of_df_in_data['key_C']

,key1,key2,key3,key4
0,5.000000,-0.050000,500000.0,3125.0000
1,2.900000,1.000000,0.0,-209.1000
2,0.000007,666.666667,17.0,3.1416
3,-19.000000,-2.000000,-1000.0,6.6000


In [33]:
dict_of_df_in_data['key_D']

,key1,key2,key3,key4
0,15.708000,-0.15708,1.570800e+06,9817.500000
1,9.110640,3.14160,0.000000e+00,-656.908560
2,0.000022,2094.40000,5.340720e+01,9.869651
3,-59.690400,-6.28320,-3.141600e+03,20.734560


In [34]:
# DiscAllTypes inputs

values_dict = {}
values_dict[f'{ns}.{sc_name}.z'] = 1.  
values_dict['doe.DiscAllTypesDoeScenario.DiscAllTypes.h'] = h_data
values_dict['doe.DiscAllTypesDoeScenario.DiscAllTypes.dict_in'] = dict_in_data
values_dict['doe.DiscAllTypesDoeScenario.DiscAllTypes.df_in'] = df_in_data
values_dict[f'{ns}.{sc_name}.weather'] =  weather_data
values_dict['doe.DiscAllTypesDoeScenario.DiscAllTypes.dict_of_dict_in'] = dict_of_dict_in_data
values_dict['doe.DiscAllTypesDoeScenario.DiscAllTypes.dict_of_df_in'] = dict_of_df_in_data
#values_dict['doe.DiscAllTypesDoeScenario.DiscAllTypes.cache_type'] = 'MemoryFullCache' #KO !! Problem with dict_in

values_dict
my_dict = exec_eng.get_anonimated_data_dict()
pd.DataFrame.from_dict(values_dict,orient='index')

,0
doe.DiscAllTypesDoeScenario.z,1.0
doe.DiscAllTypesDoeScenario.DiscAllTypes.h,"[0.0, 0.0, 0.0, 0.0]"
doe.DiscAllTypesDoeScenario.DiscAllTypes.dict_in,"{'key0': 0.0, 'key1': 0.0}"
doe.DiscAllTypesDoeScenario.DiscAllTypes.df_in,variable c2 c3 0 0.0 1.0 ...
doe.DiscAllTypesDoeScenario.weather,"cloudy, it is Toulouse ..."
doe.DiscAllTypesDoeScenario.DiscAllTypes.dict_of_dict_in,"{'key_A': {'subKey1': 0.1234, 'subKey2': 111.1..."
doe.DiscAllTypesDoeScenario.DiscAllTypes.dict_of_df_in,"{'key_C': ['key1', 'key2', 'key3', 'key4'], 'k..."


In [35]:
exec_eng.dm.set_values_from_dict(values_dict)
exec_eng.configure()

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi


# 4 Run Doe scenario

In [36]:
res = exec_eng.execute()
# res is without interest (retourn not the expected values but probably default values ) !

INFO:SoS.EE:PROCESS EXECUTION doe STARTS...
INFO:gemseo.core.doe_scenario: 
INFO:gemseo.core.doe_scenario:*** Start DOE Scenario execution ***
INFO:gemseo.core.doe_scenario:DiscAllTypesDoeScenario
   Disciplines: DiscAllTypes
   MDOFormulation: MDF
   Algorithm: None
INFO:SoS.EE.SoSDoeScenario:{'iterations': 5, 'eval_jac': False, 'n_processes': 1, 'wait_time_between_samples': 0.0, 'seed': 1, 'max_time': 0}
INFO:gemseo.algos.doe.doe_lib:Optimization problem:
   Minimize: doe.DiscAllTypesDoeScenario.o(doe.DiscAllTypesDoeScenario.z, doe.DiscAllTypesDoeScenario.DiscAllTypes.h)
   With respect to: doe.DiscAllTypesDoeScenario.z, doe.DiscAllTypesDoeScenario.DiscAllTypes.h
INFO:gemseo.algos.driver_lib:DOE sampling:  33%|███▎      | 33/100 [00:00<00:00, 883.03 it/sec, obj=1.75]


my_options
{'iterations': 5, 'eval_jac': False, 'n_processes': 1, 'wait_time_between_samples': 0.0, 'seed': 1, 'max_time': 0}


INFO:gemseo.algos.driver_lib:DOE sampling: 100%|██████████| 100/100 [00:00<00:00, 291.04 it/sec, obj=1.27]
INFO:gemseo.algos.driver_lib:Optimization result:
Objective value = 0.013274191644466958
The result is feasible.
Status: None
Optimizer message: None
Number of calls to the objective function by the optimizer: 100
INFO:gemseo.algos.driver_lib:Design space:
+--------------------------------------------+-------------+--------------------+-------------+-------+
| name                                       | lower_bound |       value        | upper_bound | type  |
+--------------------------------------------+-------------+--------------------+-------------+-------+
| doe.DiscAllTypesDoeScenario.z              |      0      | 0.0417022004702574 |      10     | float |
| doe.DiscAllTypesDoeScenario.DiscAllTypes.h |     -10     |  1.7490668861813   |      10     | float |
| doe.DiscAllTypesDoeScenario.DiscAllTypes.h |      0      | 3.617234050834533  |      10     | float |
+-----------

Remark : all options that are None are no kept and so appear with a warning (this is the case of default None options thar are not setted with not None value)

# 5 Display Doe  Scenario results

In [37]:
# retrieve discipline to check the result...
doe_disc = exec_eng.dm.get_disciplines_with_name("doe." + sc_name)[0]

In [38]:
type(doe_disc)

sos_trades_core.execution_engine.sos_doe_scenario.SoSDoeScenario

In [39]:
doe_disc.get_disc_full_name()

'doe.DiscAllTypesDoeScenario'

In [40]:
opt_P = doe_disc.opt_problem
print(opt_P)

Optimization problem:
   Minimize: doe.DiscAllTypesDoeScenario.o(doe.DiscAllTypesDoeScenario.z, doe.DiscAllTypesDoeScenario.DiscAllTypes.h)
   With respect to: doe.DiscAllTypesDoeScenario.z, doe.DiscAllTypesDoeScenario.DiscAllTypes.h


In [41]:
doe_disc_input = doe_disc.get_sosdisc_inputs()
pd.DataFrame.from_dict(doe_disc_input,orient='index')

,0
n_samples,100
algo,lhs
design_space,variable value lower_bnd upper...
formulation,MDF
objective_name,o
differentiation_method,finite_differences
algo_options,"{'iterations': 5, 'eval_jac': False, 'n_proces..."
parallel_options,"{'parallel': False, 'n_processes': 4, 'use_thr..."
eval_mode,False
eval_jac,False


In [42]:
doe_disc_output = doe_disc.get_sosdisc_outputs()

In [43]:
# doe_disc_output

# 5.1 Table of computed points

In [44]:
XY_pd= doe_disc_output['doe_ds_io']
XY_pd

design_parameters                                             \
   doe.DiscAllTypesDoeScenario.z doe.DiscAllTypesDoeScenario.DiscAllTypes.h   
                               0                                          0   
0                       9.379764                                   1.515949   
1                       8.886448                                  -1.196271   
2                       3.713714                                  -1.994758   
3                       0.041702                                   1.749067   
4                       6.954955                                  -9.316161   
..                           ...                                        ...   
95                      6.661678                                   5.719822   
96                      2.358656                                  -6.389327   
97                      3.011475                                   8.776000   
98                      4.804455                                  -2.413481   
99                      4.001988                                   2.954436   

                                 functions  
             doe.DiscAllTypesDoeScenario.o  
           1                             0  
0   5.631736                      2.985665  
1   6.523436                      2.828638  
2   4.822571                      1.182109  
3   3.617234                      0.013274  
4   9.918161                      2.213826  
..       ...                           ...  
95  7.645988                      2.120473  
96  5.297002                      0.750782  
97  9.837792                      0.958580  
98  1.131552                      1.529302  
99  9.746571                      1.273869  

[100 rows x 4 columns]

In [45]:
X_pd = XY_pd['design_parameters']
X_pd

doe.DiscAllTypesDoeScenario.z doe.DiscAllTypesDoeScenario.DiscAllTypes.h  \
                               0                                          0   
0                       9.379764                                   1.515949   
1                       8.886448                                  -1.196271   
2                       3.713714                                  -1.994758   
3                       0.041702                                   1.749067   
4                       6.954955                                  -9.316161   
..                           ...                                        ...   
95                      6.661678                                   5.719822   
96                      2.358656                                  -6.389327   
97                      3.011475                                   8.776000   
98                      4.804455                                  -2.413481   
99                      4.001988                                   2.954436   

              
           1  
0   5.631736  
1   6.523436  
2   4.822571  
3   3.617234  
4   9.918161  
..       ...  
95  7.645988  
96  5.297002  
97  9.837792  
98  1.131552  
99  9.746571  

[100 rows x 3 columns]

In [46]:
#X_pd.to_csv('X_pd.csv',index=False)

In [47]:
Y_pd = XY_pd['functions']
Y_pd

,doe.DiscAllTypesDoeScenario.o
,0
0,2.985665
1,2.828638
2,1.182109
3,0.013274
4,2.213826
...,...
95,2.120473
96,0.750782
97,0.958580


# 5.2 Optimum

In [48]:
my_optim_result = doe_disc_output['optim_result']
my_optim_result

{'x_opt': array([0.0417022 , 1.74906689, 3.61723405]),
 'f_opt': 0.013274191644466958}

In [49]:
my_optim_result['x_opt']

array([0.0417022 , 1.74906689, 3.61723405])

In [50]:
my_optim_result['f_opt']

0.013274191644466958

# 5.3 From gemseo 

In [51]:
print(opt_P.objective)
print(opt_P.constraints)
print(opt_P.differentiation_method)
print(opt_P.fd_step)

doe.DiscAllTypesDoeScenario.o(doe.DiscAllTypesDoeScenario.z, doe.DiscAllTypesDoeScenario.DiscAllTypes.h)
[]
finite_differences
1e-06


In [52]:
dataset = opt_P.export_to_dataset("dataset_name")

In [53]:
XY_pd=dataset.export_to_dataframe()
XY_pd

design_parameters                                             \
   doe.DiscAllTypesDoeScenario.z doe.DiscAllTypesDoeScenario.DiscAllTypes.h   
                               0                                          0   
0                       9.379764                                   1.515949   
1                       8.886448                                  -1.196271   
2                       3.713714                                  -1.994758   
3                       0.041702                                   1.749067   
4                       6.954955                                  -9.316161   
..                           ...                                        ...   
95                      6.661678                                   5.719822   
96                      2.358656                                  -6.389327   
97                      3.011475                                   8.776000   
98                      4.804455                                  -2.413481   
99                      4.001988                                   2.954436   

                                 functions  
             doe.DiscAllTypesDoeScenario.o  
           1                             0  
0   5.631736                      2.985665  
1   6.523436                      2.828638  
2   4.822571                      1.182109  
3   3.617234                      0.013274  
4   9.918161                      2.213826  
..       ...                           ...  
95  7.645988                      2.120473  
96  5.297002                      0.750782  
97  9.837792                      0.958580  
98  1.131552                      1.529302  
99  9.746571                      1.273869  

[100 rows x 4 columns]

In [54]:
X_pd = XY_pd['design_parameters']
X_pd

doe.DiscAllTypesDoeScenario.z doe.DiscAllTypesDoeScenario.DiscAllTypes.h  \
                               0                                          0   
0                       9.379764                                   1.515949   
1                       8.886448                                  -1.196271   
2                       3.713714                                  -1.994758   
3                       0.041702                                   1.749067   
4                       6.954955                                  -9.316161   
..                           ...                                        ...   
95                      6.661678                                   5.719822   
96                      2.358656                                  -6.389327   
97                      3.011475                                   8.776000   
98                      4.804455                                  -2.413481   
99                      4.001988                                   2.954436   

              
           1  
0   5.631736  
1   6.523436  
2   4.822571  
3   3.617234  
4   9.918161  
..       ...  
95  7.645988  
96  5.297002  
97  9.837792  
98  1.131552  
99  9.746571  

[100 rows x 3 columns]

In [55]:
Y_pd = XY_pd['functions']
Y_pd

,doe.DiscAllTypesDoeScenario.o
,0
0,2.985665
1,2.828638
2,1.182109
3,0.013274
4,2.213826
...,...
95,2.120473
96,0.750782
97,0.958580


In [56]:
import numpy as np
min(np.array(Y_pd['doe.DiscAllTypesDoeScenario.o']['0']))

0.013274191644466958

In [57]:
doe_disc.optimization_result.f_opt

0.013274191644466958

In [58]:
doe_disc.optimization_result.x_opt

array([0.0417022 , 1.74906689, 3.61723405])

In [59]:
doe_disc.print_execution_metrics()

INFO:gemseo.core.scenario:* Scenario Executions statistics *
INFO:gemseo.core.scenario:* Discipline: DiscAllTypes
INFO:gemseo.core.scenario:Executions number: 101
INFO:gemseo.core.scenario:Execution time:  0.23759249999999632 s
INFO:gemseo.core.scenario:Linearizations number: 0
INFO:gemseo.core.scenario:Total number of executions calls 101
INFO:gemseo.core.scenario:Total number of linearizations 0


In [60]:
doe_disc.get_optimum().__dict__

{'x_0': array([9.37976388, 1.51594904, 5.63173624]),
 'optimizer_name': 'lhs',
 'x_opt': array([0.0417022 , 1.74906689, 3.61723405]),
 'message': None,
 'f_opt': 0.013274191644466958,
 'constraints_values': {},
 'constraints_grad': {},
 'status': None,
 'n_obj_call': 100,
 'n_grad_call': None,
 'n_constr_call': None,
 'is_feasible': True}

# 6 The full XY_pd table

need 'MemoryFullCache' : but error when using 'MemoryFullCache' in SoSTrades 

In [61]:
#dataset = disc.cache.export_to_dataset()
#XY_pd=dataset.export_to_dataframe()

In [62]:
#XY_pd